In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Load data
df = pd.read_csv("data/insurance.csv")
X = df["age"].values
y = df["charges"].values

# Normalize
X = (X - X.mean()) / X.std()
y = (y - y.mean()) / y.std()

# Parameters
w, b = 0.0, 0.0
lr = 0.1
epochs = 300
history = []

# Training
for epoch in range(epochs):
    y_pred = w * X + b
    error = y_pred - y
    mse = np.mean(error**2)

    history.append({
        "epoch": epoch,
        "mse": mse,
        "w": w,
        "b": b,
        "y_pred": y_pred.copy()
    })

    dw = np.mean(error * X)
    db = np.mean(error)
    w -= lr * dw
    b -= lr * db

# Static range for w
w_range = np.linspace(-2, 2, 200)

# Fungsi hitung turunan numerik
def numerical_derivative(w_vals, mse_vals, w_point):
    idx = np.searchsorted(w_vals, w_point)
    if idx <= 0: idx = 1
    if idx >= len(w_vals) - 1: idx = len(w_vals) - 2
    x0, x1 = w_vals[idx - 1], w_vals[idx + 1]
    y0, y1 = mse_vals[idx - 1], mse_vals[idx + 1]
    return (y1 - y0) / (x1 - x0)

# Fungsi plot interaktif
def plot_with_true_tangent(epoch):
    record = history[epoch]
    w_curr = record["w"]
    b_curr = record["b"]

    # Recompute MSE curve for b fixed at current epoch
    mse_curve = []
    for w_val in w_range:
        y_hat = w_val * X + b_curr
        mse = np.mean((y_hat - y) ** 2)
        mse_curve.append(mse)

    # Get true slope from curve (not from training)
    mse_curr = np.mean((w_curr * X + b_curr - y)**2)
    slope = numerical_derivative(w_range, mse_curve, w_curr)

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Left: Fit line
    axes[0].scatter(X, y, label="Actual")
    axes[0].plot(X, record["y_pred"], color="red", label="Prediction")
    axes[0].set_title(f"Fit Line - Epoch {epoch}")
    axes[0].set_xlabel("Age (normalized)")
    axes[0].set_ylabel("Charges (normalized)")
    axes[0].legend()
    axes[0].grid(True)

    # Right: MSE vs w + true tangent line
    axes[1].plot(w_range, mse_curve, label="MSE Curve")
    axes[1].plot([w_curr], [mse_curr], 'ro', label="Current (w, MSE)")

    # TRUE tangent line
    w_tangent = np.linspace(w_curr - 1, w_curr + 1, 100)
    mse_tangent = slope * (w_tangent - w_curr) + mse_curr
    axes[1].plot(w_tangent, mse_tangent, 'g--', label="Tangent Line")

    # GD path so far
    past_ws = [h["w"] for h in history[:epoch+1]]
    past_mses = [np.mean((h["w"] * X + h["b"] - y)**2) for h in history[:epoch+1]]
    axes[1].plot(past_ws, past_mses, 'ro-', alpha=0.5)

    axes[1].set_title("Gradient Descent Path on MSE vs w")
    axes[1].set_xlabel("w")
    axes[1].set_ylabel("MSE")
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

# Run interactive slider
interact(plot_with_true_tangent, epoch=IntSlider(min=0, max=epochs-1, step=1, value=0));


interactive(children=(IntSlider(value=0, description='epoch', max=299), Output()), _dom_classes=('widget-inter…